In [1]:
import os
import openai
from dotenv import load_dotenv

In [2]:
load_dotenv()
print("hello")

hello


In [3]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [4]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import Document

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()


document = Document(text="\n\n".join([doc.text for doc in documents]))

In [5]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo",temperature=0.1)

In [6]:
# import os

# save_dir = 'c:/Users/swati/OneDrive/Desktop/LLM_basics/RAG/sentence_index/'
# if not os.path.exists(save_dir):
#     os.makedirs(save_dir)

In [7]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",#identifies relevent info
    save_dir="sentence_index"
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


c:\Users\swati\anaconda3\envs\Rag\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

c:\Users\swati\anaconda3\envs\Rag\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
output = sentence_window_engine.query("How to create your AI portfolio?")
output.response

'To create your AI portfolio, focus on building a collection of projects that demonstrate a progression of skills. Start by working on projects that align with your career goals and showcase your abilities effectively. Utilize a simple framework to structure your portfolio, ensuring it highlights your expertise and growth in AI. Additionally, consider using informational interviews to gather insights on the types of projects that can help you stand out to potential employers in the field of AI.'

### Feedback Functions

In [10]:
import nest_asyncio
nest_asyncio.apply()

In [11]:
from trulens_eval import OpenAI as fOpenAI

provider = fOpenAI()

### Answer Relevance

In [12]:
from trulens_eval import Feedback

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name = "Answer Relevance"
).on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


### Context Relevance

In [13]:
from trulens_eval import TruLlama

context_sec = TruLlama.select_source_nodes().node.text

In [14]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons,
             name = "Context Relevance")
             .on_input()
             .on(context_sec)
             .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [15]:
from trulens_eval import feedback,Select
from trulens_eval import (
    Feedback,
    TruLlama,
    OpenAI
)

provider = OpenAI()
f_groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(context_sec)
    .on_output()
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


### Evaluation of the RAG application

In [16]:
from trulens_eval import TruLlama
from trulens_eval import FeedbackMode

true_recorder = TruLlama(
    sentence_window_engine,
    app_id = "APP_1",
    feedbacks=[
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]
)

In [17]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [18]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?']

In [19]:
eval_questions.append("How can I be successful in AI?")
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?',
 'How can I be successful in AI?']

In [20]:
for question in eval_questions:
    with true_recorder as recording:
        sentence_window_engine.query(question)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swati\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swati\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swati\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


...

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swati\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swati\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
records,feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Groundedness,Context Relevance,Answer Relevance_calls,Groundedness_calls,Context Relevance_calls,latency,total_tokens,total_cost
0,APP_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_86a2679af1e70ce1ed06bc640f28c74c,"""What are the keys to building a career in AI?""","""Learning foundational technical skills, worki...",-,"{""record_id"": ""record_hash_86a2679af1e70ce1ed0...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-06T12:02:53.365075"", ""...",2024-06-06T12:02:58.473048,0.8,0.900000,0.85,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'source': 'Chapter 6: Building a Po...,[{'args': {'question': 'What are the keys to b...,5,540,0.000825
1,APP_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_cd9a7661d456c85eefa44e1fd8d3a6c6,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by a...",-,"{""record_id"": ""record_hash_cd9a7661d456c85eefa...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-06T12:02:58.765231"", ""...",2024-06-06T12:03:05.039722,1.0,0.533333,0.80,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'To get a project starte...,[{'args': {'question': 'How can teamwork contr...,6,706,0.001096
2,APP_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_f9a61338ea55eabbf75849726c2b876a,"""What is the importance of networking in AI?""","""Networking in AI is crucial as it can provide...",-,"{""record_id"": ""record_hash_f9a61338ea55eabbf75...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-06T12:03:05.246590"", ""...",2024-06-06T12:03:12.206091,0.9,0.575000,0.60,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'What is the hiring proce...,[{'args': {'question': 'What is the importance...,6,588,0.000935
3,APP_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_75030a9608cab86d570550470ab3079d,"""What are some good habits to develop for a su...","""Developing good habits in areas such as eatin...",-,"{""record_id"": ""record_hash_75030a9608cab86d570...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-06T12:03:12.436240"", ""...",2024-06-06T12:03:17.431059,1.0,0.750000,0.60,[{'args': {'prompt': 'What are some good habit...,"[{'args': {'source': '(For example, many arti...",[{'args': {'question': 'What are some good hab...,4,548,0.000840
4,APP_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_f78535a704721149344110d9490d4b13,"""How can altruism be beneficial in building a ...","""Helping others and aiming to lift them during...",-,"{""record_id"": ""record_hash_f78535a704721149344...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-06T12:03:17.668943"", ""...",2024-06-06T12:03:22.457076,0.8,0.000000,0.20,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'source': 'Many successful people d...,[{'args': {'question': 'How can altruism be be...,4,509,0.000774


In [22]:
import pandas as pd

pd.set_option("display.max_colwidth",None)
records[["input","output"]+feedback]

,input,output,Context Relevance,Answer Relevance,Groundedness
0,"""What are the keys to building a career in AI?""","""Learning foundational technical skills, working on projects, finding a job, and being part of a supportive community are the keys to building a career in AI.""",0.85,0.8,0.900000
1,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by allowing individuals to leverage the diverse skills and insights of their colleagues. Collaborating with others can lead to a more comprehensive understanding of complex problems, innovative solutions, and effective project management. Additionally, working in a team can foster a supportive environment that encourages continuous learning, hard work, and a shared commitment to achieving common goals.""",0.80,1.0,0.533333
2,"""What is the importance of networking in AI?""","""Networking in AI is crucial as it can provide valuable insights, guidance, and opportunities for individuals looking to advance in the field. By connecting with professionals who have experience in AI, individuals can gain knowledge about the industry, potential career paths, and current trends. Networking also allows individuals to build relationships with others in the field, which can lead to mentorship, collaboration on projects, and even job opportunities. Additionally, networking can help individuals stay informed about the evolving nature of AI and understand how different companies may define roles within the field.""",0.60,0.9,0.575000
3,"""What are some good habits to develop for a successful career?""","""Developing good habits in areas such as eating, exercise, sleep, personal relationships, work, learning, and self-care can help individuals move forward in their careers while maintaining their health.""",0.60,1.0,0.750000
4,"""How can altruism be beneficial in building a career?""","""Helping others and aiming to lift them during one's career journey can lead to better outcomes for oneself.""",0.20,0.8,0.000000
5,"""What is imposter syndrome and how does it relate to AI?""","""Imposter syndrome is a psychological pattern where individuals doubt their accomplishments and have a persistent fear of being exposed as a fraud, despite evidence of their competence. In the context of AI, newcomers to the field may experience imposter syndrome, feeling like they do not truly belong in the AI community even if they have achieved success. This can be a common experience for individuals in AI, including accomplished figures like Sheryl Sandberg and Michelle Obama, and it is important to address and overcome these feelings to continue growing in the field.""",0.95,0.8,0.866667
6,"""Who are some accomplished individuals who have experienced imposter syndrome?""","""Former Facebook COO Sheryl Sandberg, U.S. first lady Michelle Obama, actor Tom Hanks, and Atlassian co-CEO Mike Cannon-Brookes are some accomplished individuals who have experienced imposter syndrome.""",0.95,1.0,0.500000
7,"""What is the first step to becoming good at AI?""","""The first step to becoming good at AI is to learn foundational technical skills.""",0.80,0.9,0.500000
8,"""What are some common challenges in AI?""","""Common challenges in AI include the evolving nature of technologies, the need to continuously update skills due to rapid changes in technology, difficulties in project management due to the iterative nature of AI projects, collaborating with stakeholders who may lack expertise in AI, uncertainties in defining AI job roles and required skills, and the presence of imposter syndrome among individuals in the AI community.""",0.50,0.8,0.500000
9,"""Is it normal to find parts of AI challenging?""","""It is normal to find parts of AI challenging, as even experienced individuals in the field have faced difficulties with various aspects of artificial intelligence.""",NaN,0.9,NaN


In [23]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Answer Relevance,Groundedness,latency,total_cost
app_id,,,,,
APP_1,0.694444,0.89,0.569444,4.909091,0.000918


In [24]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swati\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swati\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swati\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

Dashboard started at http://192.168.0.103:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]